<a href="https://colab.research.google.com/github/MerlijnvEmmerik/Mijnrel/blob/master/Getting_fisherM_week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import scipy.special as special
from sympy import *
import numpy as np
import math
import cmath
from scipy.integrate import quad    

M1 = 4*10**40 
M2 = 4*10**40
Mtot = M1 + M2
etha = M1+M2 / Mtot**2
Mchirp = etha**(3/5)
phic = 1.0
tc = 1.0
SNR = 10.0
freq = 1.0


G = 6.67*10**(-11)    # (m**3)*(kg**-1)*(s**-2)
c = 3*10**8    # m*(s**-1)

#defining which detector##########################################
detector = "Ligo"

if detector == "AdvancedLigo":
    freq0 = 215
    freqs = 20
    S0 = 6 * 10**(-49)
    def S_f(freq):
        if (freq >= freqs):
            return S0*((freq/freq0)**(-4.14) - 5 * (freq/freq0)**(-2) +
                   ((111 * (1 - (freq/freq0)**2 + ((freq/freq0)**4) / 2) / (1 + ((freq/freq0)**2) / 2))))
        else: 
            return float('inf')
    
elif detector == "Ligo":
    freq0 = 150
    freqs = 40
    S0 = 9 * 10**(-46)    
    def S_f(freq):
        if (freq >= freqs):
            return S0*((4.49*(freq/freq0))**(-56)+0.16*(freq/freq0)**(-4.52)+ 0.52 + 0.32*(freq/freq0**2))
        else: 
            return float('inf')


elif detector == "Virgo":
    freq0 = 500
    freqs = 20
    S0 = 3.24*10**(-46)
    def S_f(freq):
        if (freq >= freqs):
            return S0*((6.23*(freq/freq0))**(-5)+2*(freq/freq0)**(-1)+ 1 + (freq/freq0**2))
        else: 
            return float('inf')
    


#list of variables for i and j#########################################################################################

var = [phic, etha, Mchirp, tc]

#defining v, S(f) and amplitude ################################################################################## 

def v(etha, Mchirp, freq):
  outcome = np.pi*(((etha**(-3/5))*Mchirp)*freq*G/c**3)**(1/3)
  print("v :"+str(type(outcome)))
  return outcome 
       
def Ampl(freq, SNR, freq0, freqs):
  Sf = S_f(freq)  
  function = (freq**(-7/6))/Sf
  integration = integrate(lambda freq: function, freqs, freq0, 100)
  outcome = np.sqrt(SNR/(4)*integration)
  print("Ampl :"+str(type(outcome)))
  return outcome


#defining alphas ######################################################################################################################
alpha0 = 1 
alpha1 = 0
  
def alpha2(etha):
  outcome = (20/9)*((743/336) + (11/4)*etha)
  print("alpha2 :"+str(type(outcome)))
  return outcome 

alpha3 = -16*np.pi

#defining phase ########################################################################################################################
def phase(freq, tc, phic, etha, Mchirp, alpha0, alpha1, alpha3):
  v_New = v(etha, Mchirp, freq)
  alpha2_New = alpha2(etha)
  outcome = 2*(np.pi)*freq*tc - phic - ((np.pi)/4) + (3/(128*etha*(v_New**5)))*(alpha0*1 + alpha1*(v_New**1) + alpha2_New*(v_New**2) + alpha3*(v_New**3))
  print("phase :"+str(type(outcome)))
  return outcome

#defining gravitational wave Fourier transform #######################################################################################
def h_f(freq, tc, etha, Mchirp, phic, alpha0, alpha1, alpha3, SNR, freq0, freqs):
  Ampl_New = Ampl(freq, SNR, freq0, freqs)
  phase_New = phase(freq, tc, phic, etha, Mchirp, alpha0, alpha1, alpha3)
  outcome = Ampl_New*(freq**(-7/6))*(np.cos(phase_New) + 1*np.sin(phase_New))
  print("h_f :"+str(type(outcome)))
  return outcome

#defining derrivative and integration ###############################################################################################################


def derive(function, x):
    h=1./1000.
    rise = function(x + h) - function(x)
    run = h
    slope = rise/run
    print("derive :"+str(type(slope)))
    return slope


def integrate(f, low, high, N):
    h = (high - low)/N
    S = 0.5*f(low) + f(high)
    for i in range(1,N):
        S += f(low + i*h)
    print("integrate :"+str(type(S)))    
    return S


#making the Fisher Matrix#####################################################################################
def Fish_M(freq, tc, etha, Mchirp, phic, alpha0, alpha1, alpha3, SNR, freq0, freqs):
    var = [phic, etha, Mchirp, tc]
    Sf = S_f(freq)
    h_f_New = h_f(freq, tc, etha, Mchirp, phic, alpha0, alpha1, alpha3, SNR, freq0, freqs)
    matrix = []
    for k in range(len(var)):
        matrix.append([])
        for i in var:
            dhf_di = derive(h_f_New, i)
        for j in var:
            dhf_dj = derive(h_f_New, j) 
            a = dhf_di*dhf_dj
            function_fisher = a / Sf
            b = 4* integrate(lambda freq: function_fisher, freqs, freq0, 100) 
            matrix[-1].append(b)
    return matrix 

#inverting matrix and getting diagonal elements###################################################################
def get_diagonal_inverse(matrix):
  a = np.linalg.inv(matrix) 
  New_Matrix = []
  for i in range (len(a)):
    New_Matrix.append(np.sqrt(a[i][i]))
  return New_Matrix

